## Experimento 5. 
## Análisis del envío de remesas o acumulación de ahorros durante el último viaje migratorio

Selección y limpieza de datos a partir de las tablas <b>MIG</b>, <b>LIFE</b> y <b>COMMUN</b>.

Para este experimento se considera solamente información pertinente al último viaje migratorio realizado, creando una etiqueta de clase a partir de los atributos 'remit1'...'remit5' y 'savretrn'. 

Así, la etiqueta de clase puede tomar dos valores posibles:<br>
>1: Sí (Si la persona envió remesas o regresó con ahorros a México)<br>0: No (Si no envió remesas ni regresó con ahorros)<br>
---

Bibliotecas utilizadas

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import cpi #Paquete para ajustar salarios y precios en USD de acuerdo a los índices de inflación del último año que se realizó la encuesta	
import os
import time
from multiprocessing import Pool, cpu_count

/tmp/ipykernel_252595/383984047.py:4: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  import cpi #Paquete para ajustar salarios y precios en USD de acuerdo a los índices de inflación del último año que se realizó la encuesta


Rutas de acceso y almacenamiento de los archivos generados

In [2]:
#Ruta para leer el archivo a procesar
path_to_data_core_files = os.path.join('../'*3, 'Bases de datos', 'Princeton', 'Mexican Migration Project (MMP)', 'CSV', 'core_files')

path_to_data_supplemental_files = os.path.join('../'*3, 'Bases de datos', 'Princeton', 'Mexican Migration Project (MMP)', 'CSV', 'supplemental_files')


#Ruta para almacenar el dataset procesado
path_to_store = os.path.join('..', 'datasets','remesas_clasificacion')

Atributos a leer de cada archivo de datos

In [3]:
#Selección de atributos relacionados solamente al  último viaje migratorio
id_atts = ['country', 'commun', 'surveypl', 'surveyyr', 'hhnum'] #Información de identificación

mig_atts = id_atts + [
  #Información general
  'usyrl',
  #Información demográfica	
  'age','sex','usmarl','usdocl',
  #Capital humano
  'edyrs',
  #Características socioeconómicas
  'hrwage', 
  #Características migratorias
  'usdurl',
  #Lazos sociales
  'paistrip', 'reltrip', 'sport', 'social',
  #Características sobre remesas
  'remit', 'savretrn']

life_atts = id_atts + [
    #Información del año persona
    'year', 'age', 
    #Información demográfica	
    'minors',
    #Características socioeconómicas
    #Propiedades del hogar
    'land', 'property','business'
  ]

#Lazos familiares
spouse_atts = id_atts + [
    #Año
    'year',
    #La o el cónyuge está en México
    'inus'
  ]

house_atts = id_atts + [
  #Número de hijos
  'sons',
  #Número de hijas
  'daus'
]

commun_atts = [
    #Identificador de comunidad
    'COMMUN',
    #Nivel de urbanización
    'METROCAT'
    #Evaluar si incluir el nivel de desarrollo de la comunidad
]

natlyear_atts =  [ 
    #Año
    'year',
    #Tasa de desempleo en EEUU  
    'usunemp'
  ]

Lectura de archivos de datos

In [4]:
mig_data_file = os.path.join(path_to_data_core_files, 'mig174.csv')
life_file = os.path.join(path_to_data_core_files, 'life174.csv')
commun_file = os.path.join(path_to_data_supplemental_files, 'commun174.csv')
natlyear_file = os.path.join(path_to_data_supplemental_files, 'natlyear2019.csv')
spouse_file = os.path.join(path_to_data_core_files, 'spouse174.csv')
house_file = os.path.join(path_to_data_core_files, 'house174.csv')

#Lectura de los archivos
mig = pd.read_csv(mig_data_file, na_values=['9999',' '], usecols=mig_atts)
life = pd.read_csv(life_file, na_values=['9999',' '], usecols=life_atts)
commun = pd.read_csv(commun_file, na_values=['9999',' '], usecols=commun_atts)
natlyear = pd.read_csv(natlyear_file, na_values=['9999',' '], usecols=natlyear_atts)
spouse = pd.read_csv(spouse_file, na_values=['9999',' '], usecols=spouse_atts)
house = pd.read_csv(house_file, na_values=['9999',' '], usecols=house_atts)

#Eliminación de comunidades de 1 a 71 (No alpica usmarl)
mig = mig[mig['commun'] > 71]
life = life[life['commun'] > 71]
commun = commun[commun['COMMUN'] > 71]
spouse = spouse[spouse['commun'] > 71]
house = house[house['commun'] > 71]

Obtención de características del archivo life

In [5]:
data = pd.merge(mig,life,left_on=['country','commun','surveypl','surveyyr','hhnum','usyrl'],right_on=['country','commun','surveypl','surveyyr','hhnum','year'],how='inner',suffixes=('_mig','_life'))
data = pd.merge(data,commun,left_on=['commun'],right_on=['COMMUN'],how='inner')
data.rename(columns={'age_life':'age'},inplace=True)
data.drop(columns=['year'],inplace=True)

data = pd.merge(data,natlyear,left_on=['usyrl'],right_on=['year'],how='inner', suffixes=('_data','_natlyear'))


data = pd.merge(data,spouse,left_on=['country','commun','surveypl','surveyyr','hhnum', 'usyrl'],right_on=['country','commun','surveypl','surveyyr','hhnum','year'],how='inner',suffixes=('_data','_spouse'))

data = pd.merge(data,house,left_on=['country','commun','surveypl','surveyyr','hhnum'],right_on=['country','commun','surveypl','surveyyr','hhnum'],how='inner',suffixes=('_data','_house'))

data.drop(columns=['country','commun','surveypl','surveyyr','hhnum','age_mig','year_spouse','year_data','COMMUN'],inplace=True)
#data.drop(columns=['country','commun','surveypl','surveyyr','hhnum','age_mig','COMMUN', 'year'],inplace=True)

#data.reset_index(drop=True,inplace=True)
data.columns

Index(['sex', 'edyrs', 'usyrl', 'usmarl', 'usdurl', 'usdocl', 'reltrip',
       'paistrip', 'sport', 'social', 'hrwage', 'remit', 'savretrn', 'age',
       'minors', 'land', 'property', 'business', 'METROCAT', 'usunemp', 'inus',
       'sons', 'daus'],
      dtype='object')

In [6]:
data = data[(data['remit'].isna() == False) & (data['savretrn'].isna() == False)] #Eliminación de personas que no enviaron remesas ni ahorros
data = data.dropna(subset=['usyrl']) #Eliminación de personas cuyo último año de viaje a EEUU es desconocido
data.reset_index(drop=True,inplace=True)
data

,sex,edyrs,usyrl,usmarl,usdurl,usdocl,reltrip,paistrip,sport,social,...,age,minors,land,property,business,METROCAT,usunemp,inus,sons,daus
0,1,9.0,1992.0,2.0,12,8.0,2.0,1.0,2.0,2.0,...,20,0.0,0,0,0,2,0.075,0,0,1
1,1,6.0,1999.0,1.0,7,1.0,1.0,1.0,2.0,2.0,...,23,1.0,0,2,0,2,0.042,0,1,0
2,1,6.0,1972.0,2.0,6,8.0,1.0,2.0,2.0,1.0,...,16,0.0,0,0,0,2,0.056,0,0,2
3,1,3.0,1981.0,1.0,4,8.0,1.0,2.0,2.0,2.0,...,35,5.0,0,0,0,2,0.076,0,1,2
4,1,3.0,1976.0,1.0,18,8.0,2.0,1.0,1.0,1.0,...,33,2.0,0,1,1,2,0.077,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,1,3.0,1981.0,1.0,12,8.0,1.0,2.0,NaN,NaN,...,21,0.0,0,1,0,4,0.076,0,0,0
2264,1,4.0,2004.0,1.0,60,8.0,2.0,1.0,2.0,2.0,...,63,1.0,1,0,0,4,0.055,0,1,0
2265,1,6.0,2016.0,1.0,24,8.0,2.0,1.0,2.0,2.0,...,31,4.0,1,1,0,4,0.053,0,2,2
2266,1,6.0,2007.0,1.0,36,8.0,1.0,2.0,1.0,1.0,...,37,3.0,1,1,0,4,0.046,0,0,0


In [7]:
#Ajuste de los salarios en USD de acuerdo al índice de inflación del último año que se realizó la encuesta
#utilizando la librería cpi
data['usyrl'] = data['usyrl'].astype(int)
adjustment_year = 2019
#data[f'hrwage_adj_{max_survey_year}'] = data.apply(lambda x: cpi.inflate(x.hrwage, x.usyrl, to=max_survey_year), axis=1)
#Utilizando multiprocessing para paralelizar el proceso de ajuste
results = []

print(f'Ajustando salarios y gastos en USD de acuerdo al índice de inflación del último año que se realizó la encuesta. ({adjustment_year})')
tick = time.time()
with Pool(processes=cpu_count()) as pool:
    #pool.starmap recibe una función y una lista de iterables con los argumentos de la función
    #pool.starmap(funcion, zip(arg1, arg2, arg3, ...)) -> funcion(arg1[0], arg2[0], arg3[0]), funcion(arg1[1], arg2[1], arg3[1]), ...
    results = pool.starmap(cpi.inflate, zip(data['hrwage'], data['usyrl'], [adjustment_year]*len(data)))
tock = time.time()
print(f'completado en {(tock-tick)/60:0.2f} mins.')
data[f'hrwage_adj_{adjustment_year}'] = results
#data = pd.concat([data, *results], axis=1)
#data['usyrl'] = data['usyrl'].astype('category')
data.drop(columns=['hrwage'], inplace=True)
data

Ajustando salarios y gastos en USD de acuerdo al índice de inflación del último año que se realizó la encuesta. (2019)
completado en 0.75 mins.


,sex,edyrs,usyrl,usmarl,usdurl,usdocl,reltrip,paistrip,sport,social,...,minors,land,property,business,METROCAT,usunemp,inus,sons,daus,hrwage_adj_2019
0,1,9.0,1992,2.0,12,8.0,2.0,1.0,2.0,2.0,...,0.0,0,0,0,2,0.075,0,0,1,8.199975
1,1,6.0,1999,1.0,7,1.0,1.0,1.0,2.0,2.0,...,1.0,0,2,0,2,0.042,0,1,0,NaN
2,1,6.0,1972,2.0,6,8.0,1.0,2.0,2.0,1.0,...,0.0,0,0,0,2,0.056,0,0,2,18.348589
3,1,3.0,1981,1.0,4,8.0,1.0,2.0,2.0,2.0,...,5.0,0,0,0,2,0.076,0,1,2,6.440644
4,1,3.0,1976,1.0,18,8.0,2.0,1.0,1.0,1.0,...,2.0,0,1,1,2,0.077,0,3,0,15.725826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,1,3.0,1981,1.0,12,8.0,1.0,2.0,NaN,NaN,...,0.0,0,1,0,4,0.076,0,0,0,NaN
2264,1,4.0,2004,1.0,60,8.0,2.0,1.0,2.0,2.0,...,1.0,1,0,0,4,0.055,0,1,0,4.060196
2265,1,6.0,2016,1.0,24,8.0,2.0,1.0,2.0,2.0,...,4.0,1,1,0,4,0.053,0,2,2,8.521651
2266,1,6.0,2007,1.0,36,8.0,1.0,2.0,1.0,1.0,...,3.0,1,1,0,4,0.046,0,0,0,2.466042


In [8]:
pre_irca = [1 if d >= 1965 and d <= 1986 else 0 for d in data['usyrl']]
irca = [1 if d >= 1987 and d <= 1996 else 0 for d in data['usyrl']]
iirira = [1 if d >= 1997 and d <= 2015 else 0 for d in data['usyrl']]

pais_pre_irca = [1 if d == 1 and p == 1 else 0 for (d,p) in zip(data['paistrip'],pre_irca)]
pais_irca = [1 if d == 1 and p == 1 else 0 for (d,p) in zip(data['paistrip'], irca)]
pais_iirira = [1 if d == 1 and p == 1 else 0 for (d,p) in zip(data['paistrip'],iirira)]

data['pre_irca'] = pre_irca
data['irca'] = irca
data['iirira'] = iirira
data['pais_pre_irca'] = pais_pre_irca
data['pais_irca'] = pais_irca
data['pais_iirira'] = pais_iirira

data.drop(columns=['usyrl'],inplace=True)

data

,sex,edyrs,usmarl,usdurl,usdocl,reltrip,paistrip,sport,social,remit,...,inus,sons,daus,hrwage_adj_2019,pre_irca,irca,iirira,pais_pre_irca,pais_irca,pais_iirira
0,1,9.0,2.0,12,8.0,2.0,1.0,2.0,2.0,250.0,...,0,0,1,8.199975,0,1,0,0,1,0
1,1,6.0,1.0,7,1.0,1.0,1.0,2.0,2.0,500.0,...,0,1,0,NaN,0,0,1,0,0,1
2,1,6.0,2.0,6,8.0,1.0,2.0,2.0,1.0,0.0,...,0,0,2,18.348589,1,0,0,0,0,0
3,1,3.0,1.0,4,8.0,1.0,2.0,2.0,2.0,80.0,...,0,1,2,6.440644,1,0,0,0,0,0
4,1,3.0,1.0,18,8.0,2.0,1.0,1.0,1.0,100.0,...,0,3,0,15.725826,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,1,3.0,1.0,12,8.0,1.0,2.0,NaN,NaN,200.0,...,0,0,0,NaN,1,0,0,0,0,0
2264,1,4.0,1.0,60,8.0,2.0,1.0,2.0,2.0,100.0,...,0,1,0,4.060196,0,0,1,0,0,1
2265,1,6.0,1.0,24,8.0,2.0,1.0,2.0,2.0,0.0,...,0,2,2,8.521651,0,0,0,0,0,0
2266,1,6.0,1.0,36,8.0,1.0,2.0,1.0,1.0,1000.0,...,0,0,0,2.466042,0,0,1,0,0,0


In [9]:
target = ((data['remit'] == 0.0) | (data['savretrn'] == 0.0)).astype(int).rename('target')
data = data.drop(columns=['remit', 'savretrn']) #Eliminación de los atributos 'remit' y 'savretrn'
data = pd.concat([data, target], axis=1)
data

,sex,edyrs,usmarl,usdurl,usdocl,reltrip,paistrip,sport,social,age,...,sons,daus,hrwage_adj_2019,pre_irca,irca,iirira,pais_pre_irca,pais_irca,pais_iirira,target
0,1,9.0,2.0,12,8.0,2.0,1.0,2.0,2.0,20,...,0,1,8.199975,0,1,0,0,1,0,0
1,1,6.0,1.0,7,1.0,1.0,1.0,2.0,2.0,23,...,1,0,NaN,0,0,1,0,0,1,0
2,1,6.0,2.0,6,8.0,1.0,2.0,2.0,1.0,16,...,0,2,18.348589,1,0,0,0,0,0,1
3,1,3.0,1.0,4,8.0,1.0,2.0,2.0,2.0,35,...,1,2,6.440644,1,0,0,0,0,0,0
4,1,3.0,1.0,18,8.0,2.0,1.0,1.0,1.0,33,...,3,0,15.725826,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,1,3.0,1.0,12,8.0,1.0,2.0,NaN,NaN,21,...,0,0,NaN,1,0,0,0,0,0,0
2264,1,4.0,1.0,60,8.0,2.0,1.0,2.0,2.0,63,...,1,0,4.060196,0,0,1,0,0,1,0
2265,1,6.0,1.0,24,8.0,2.0,1.0,2.0,2.0,31,...,2,2,8.521651,0,0,0,0,0,0,1
2266,1,6.0,1.0,36,8.0,1.0,2.0,1.0,1.0,37,...,0,0,2.466042,0,0,1,0,0,0,0


In [10]:
data.columns

Index(['sex', 'edyrs', 'usmarl', 'usdurl', 'usdocl', 'reltrip', 'paistrip',
       'sport', 'social', 'age', 'minors', 'land', 'property', 'business',
       'METROCAT', 'usunemp', 'inus', 'sons', 'daus', 'hrwage_adj_2019',
       'pre_irca', 'irca', 'iirira', 'pais_pre_irca', 'pais_irca',
       'pais_iirira', 'target'],
      dtype='object')

In [11]:
data.to_csv(os.path.join(path_to_store, 'remesas_clasificacion.csv'), index=False)

In [12]:
cat_atts = ['sex', 'usmarl', 'usdocl', 'reltrip', 'paistrip', 
            'sport', 'social', 'inus', 'land', 'property', 'business', 'METROCAT', 'pre_irca', 'irca', 'iirira','pais_pre_irca', 'pais_irca', 'pais_iirira']
print(cat_atts)

['sex', 'usmarl', 'usdocl', 'reltrip', 'paistrip', 'sport', 'social', 'inus', 'land', 'property', 'business', 'METROCAT', 'pre_irca', 'irca', 'iirira', 'pais_pre_irca', 'pais_irca', 'pais_iirira']


In [13]:
print([a for a in data.columns if a not in cat_atts])

['edyrs', 'usdurl', 'age', 'minors', 'usunemp', 'sons', 'daus', 'hrwage_adj_2019', 'target']


In [14]:
len(data)

2268

In [15]:
len(data.columns)

27